## Overview {#sec-overview}

This document describes the SeaBee annotation workflow developed and tested using the **Runde/Remøy** dataset during December 2022. 

The current workflow is preliminary and should become smoother once integrated with Sigma2. The aim of these notes is to help new users get started with annotation using ArcGIS Pro and to avoid common mistakes/pitfalls.

## Workflow {#sec-workflow}

### Install ArcGIS Pro {#sec-install-arcgis}

Follow the steps below to download and install ArcGIS Pro on your local machine.

 1. Contact Jan Karud to obtain login details for ArcGIS Online. You need access to **ArcGIS Pro** with the **Image Analyst** extension. Login [here](https://www.arcgis.com/index.html).
 
 2. [Login to ArcGIS Online](https://www.arcgis.com/index.html) and, under your profile, choose `Settings > Licenses`. Check that Image Analyst is available to you under `ArcGIS Pro extensions`, then click the link to `Download ArcGIS Pro` (@fig-download-arcgis-pro).
 
 3. Obtain and administrator password from IT-Vakt and run the installer. You should then be able to start ArcGIS Pro and login to the application using your ArcGIS Online credentials.
 
![Download ArcGIS Pro](./images/download_arcgis_pro.png){#fig-download-arcgis-pro}

### Setup project {#sec-setup-project}

The next step is to create a new project within ArcGIS and add SeaBee data to it.

 1. In ArcGIS Pro, create a new `Map` project. The `Name` should describe the mission/area you're annotating and `Location` should be an existing local folder on your PC.
 
:::{.callout-tip}
ArcGIS may run very slowly if you choose a network folder or a folder that is synchronised to an external server (e.g. DropBox, OneDrive, GoogleDrive).
:::

 2. Within your project folder, create three new subfolders: `class_definitions`, `vector` and `raster`. This can either be done using the `Catalog` pane in ArcGIS or using Windows File Explorer.
 
 3. Download the mission datasets and add them to the relevant subfolders. As a minimum, you will need a **class definition file** and a **georeferenced orthomosaic**. Optionally, you may include a **ground truth dataset**, a **region of interest** (ROI) file and any **pre-existing annotation** for your area of interest. See @sec-class-definitions to @sec-roi for details.
 
:::{.callout-note}
At present, most mission data is hosted on OneDrive/Sharepoint, which means it must be downloaded locally for use with ArcGIS. For large files, this can be slow. Eventually, most datasets should be hosted on Sigma2 and exposed as web mapping services (WMS). This will make it possible to add the orthomosaics to your map without having to download large volumes of raw imagery.
:::
 
### Class definition files {#sec-class-definitions}

ArcGIS Pro supports 

### Orthomosaics {#sec-mosaics}

### Ground truth data {#sec-ground-truth}

### Region of interest {#sec-roi}